In [2]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import html
import numpy as np

In [2]:
df = pd.read_csv('../data_collecting/raw_data_with_loc.csv', lineterminator='\n')
print(len(df))
df.head()

1320057


,tweetId,author_id_x,tweet,lang,created_at,author_id_y,user_loc
0,1514430374834851843,1461676693362556928,GOD LOVES YOU RUSSIA. YOUR PRAYERS ARE HEARD❤️,en,2022-04-14 02:28:32+00:00,1461676693362556928,NaN
1,1514430374134362115,1103963717178400768,🌧️ Western disturbance is likely to bring in s...,en,2022-04-14 02:28:32+00:00,1103963717178400768,"Bonn, Germany"
2,1514430373522030592,1499844277890867207,@atrupar He didn’t fucking rebuild NATO. What ...,en,2022-04-14 02:28:32+00:00,1499844277890867207,NaN
3,1514430372779544580,1072213959208525830,Nato ! Sorry Moscow! https://t.co/pkmq9Lon7d,en,2022-04-14 02:28:32+00:00,1072213959208525830,Brazil
4,1514430371991064578,193866394,Guthrie/Logan County: FROST ADVISORY IN EFFECT...,en,2022-04-14 02:28:32+00:00,193866394,"Guthrie, OK"


In [3]:
df['lang'].value_counts()

en     1031655
de       87596
es       34011
und      32628
fr       31674
        ...   
hy           4
my           4
km           2
bo           2
dv           1
Name: lang, Length: 64, dtype: int64

In [23]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [24]:
from data_cleaning import clean_tweet,remove_stopwords, deEmojify, isLocation, getLocation
from translation.google_translate import translate_text
from model.huggingface import getLabels

In [6]:
df = df.drop(columns=['author_id_y'])
df = df.rename(columns={'author_id_x' : 'author_id'})
df.head()

,tweetId,author_id,tweet,lang,created_at,user_loc
0,1514430374834851843,1461676693362556928,GOD LOVES YOU RUSSIA. YOUR PRAYERS ARE HEARD❤️,en,2022-04-14 02:28:32+00:00,NaN
1,1514430374134362115,1103963717178400768,🌧️ Western disturbance is likely to bring in s...,en,2022-04-14 02:28:32+00:00,"Bonn, Germany"
2,1514430373522030592,1499844277890867207,@atrupar He didn’t fucking rebuild NATO. What ...,en,2022-04-14 02:28:32+00:00,NaN
3,1514430372779544580,1072213959208525830,Nato ! Sorry Moscow! https://t.co/pkmq9Lon7d,en,2022-04-14 02:28:32+00:00,Brazil
4,1514430371991064578,193866394,Guthrie/Logan County: FROST ADVISORY IN EFFECT...,en,2022-04-14 02:28:32+00:00,"Guthrie, OK"


In [7]:
df['clean_tweet'] = [clean_tweet(tw) for tw in df['tweet']]

In [8]:
print(len(df))
df = df.dropna()
print(len(df))

1320057
763717


In [9]:
# clean user location
df['user_loc'] = df['user_loc'].apply(lambda row :deEmojify(row))
df = df[~df.user_loc.str.contains('#')] 
df['user_loc'] =df['user_loc'].apply(lambda x: re.sub(r'http\S+', '', x)) #return na value if list is empty

In [5]:
# extract user location ( spacy and get location )
df['extracted_user_loc'] = df['user_loc'].apply(lambda x: isLocation(x) if isLocation(x) else None)
df.to_csv('extracted_user.csv', index=False)

NameError: name 'df' is not defined

In [5]:
translated_df = pd.read_csv('../app/static/files/output/translated_data.csv')
translated_df = translated_df.drop(columns=['Unnamed: 0'])
translated_df.head()

,tweetId,author_id,tweet,lang,created_at,user_loc,clean_tweet,extracted_user_loc,translated_text
0,1514430374134362115,1103963717178400768,🌧️ Western disturbance is likely to bring in s...,en,2022-04-14 02:28:32+00:00,"Bonn, Germany",western disturbance is likely to bring in s...,Germany,western disturbance is likely to bring in s...
1,1514430372779544580,1072213959208525830,Nato ! Sorry Moscow! https://t.co/pkmq9Lon7d,en,2022-04-14 02:28:32+00:00,Brazil,nato sorry moscow,Brazil,nato sorry moscow
2,1514430371898834951,336002320,Bravo Zelenskyy! 👏👏👏👏👏\nUkraine's Zelenskiy m...,en,2022-04-14 02:28:32+00:00,"Orlando, FL",bravo zelenskyy ukraines zelenskiy moc...,Orlando,bravo zelenskyy ukraines zelenskiy moc...
3,1514430371538116612,1511956417737068546,Two Ukrainian-made anti-ship ‘Neptune’ class m...,en,2022-04-14 02:28:31+00:00,"Washington, DC",two ukrainian made anti ship neptune class m...,DC,two ukrainian made anti ship neptune class m...
4,1514430366567878657,585554619,"@RBMEllis @kellykreads Putin is "" super late t...",en,2022-04-14 02:28:30+00:00,"Scranton , PA",putin is super late term aborting ukraine ...,PA,putin is super late term aborting ukraine ...


In [17]:
print(len(translated_df))
translated_df = translated_df.dropna()
print(len(translated_df))

440535
440535


In [6]:
# one day data 
translated_df['created_at'] = pd.to_datetime(translated_df['created_at'])
translated_df['created_at'] = translated_df['created_at'].dt.strftime('%Y-%m-%d')
translated_df.head()

,tweetId,author_id,tweet,lang,created_at,user_loc,clean_tweet,extracted_user_loc,translated_text
0,1514430374134362115,1103963717178400768,🌧️ Western disturbance is likely to bring in s...,en,2022-04-14,"Bonn, Germany",western disturbance is likely to bring in s...,Germany,western disturbance is likely to bring in s...
1,1514430372779544580,1072213959208525830,Nato ! Sorry Moscow! https://t.co/pkmq9Lon7d,en,2022-04-14,Brazil,nato sorry moscow,Brazil,nato sorry moscow
2,1514430371898834951,336002320,Bravo Zelenskyy! 👏👏👏👏👏\nUkraine's Zelenskiy m...,en,2022-04-14,"Orlando, FL",bravo zelenskyy ukraines zelenskiy moc...,Orlando,bravo zelenskyy ukraines zelenskiy moc...
3,1514430371538116612,1511956417737068546,Two Ukrainian-made anti-ship ‘Neptune’ class m...,en,2022-04-14,"Washington, DC",two ukrainian made anti ship neptune class m...,DC,two ukrainian made anti ship neptune class m...
4,1514430366567878657,585554619,"@RBMEllis @kellykreads Putin is "" super late t...",en,2022-04-14,"Scranton , PA",putin is super late term aborting ukraine ...,PA,putin is super late term aborting ukraine ...


In [22]:
on13 = translated_df.loc[translated_df['created_at'] == '2022-04-13']
print(len(on13))

302057


In [25]:
on13['lang'].value_counts()

en     251839
de      16397
fr       6267
es       5859
und      4822
it       2969
sv       1657
in       1299
ja       1226
pl       1225
pt       1116
fi       1003
tr        952
nl        650
tl        647
hi        626
ru        604
ro        286
uk        258
cs        246
da        234
ca        217
zh        204
el        181
et        167
ar        148
ur        125
vi        114
no         76
ht         57
th         49
ta         47
ko         47
bn         43
mr         43
lv         36
fa         36
sl         34
gu         33
te         28
iw         20
cy         18
lt         16
ml         15
ckb        14
hu         14
bg         14
sr         12
or         11
eu         11
is          8
kn          7
ne          7
ps          6
ka          5
hy          4
si          3
km          2
pa          2
my          1
Name: lang, dtype: int64

In [7]:
on12 = translated_df.loc[translated_df['created_at'] == '2022-04-12']
print(len(on12))

117132


In [26]:
on12['lang'].value_counts()

en     89654
de      9517
es      4079
fr      3759
und     2636
it      1377
tr       958
pt       727
pl       528
in       487
sv       477
ja       328
nl       328
fi       322
ru       279
tl       196
ur       184
ro       179
hi       163
da       120
ca       116
cs       110
uk       102
et        78
ar        78
zh        58
no        55
el        37
ht        32
lv        20
vi        18
ko        15
fa        14
lt        12
th        11
bg        11
cy        10
ckb       10
sl        10
mr         7
bn         7
iw         6
hu         4
ta         3
eu         3
kn         2
sr         2
is         1
ps         1
gu         1
Name: lang, dtype: int64

In [24]:
on14 = translated_df.loc[translated_df['created_at'] == '2022-04-14']
print(len(on14))

21346


In [ ]:
df = getLocation(on12)
df.to_csv('location_user.csv', index=False)

In [39]:
df.head()

,tweetId,author_id,tweet,lang,created_at,user_loc,clean_tweet,extracted_user_loc,translated_text,latitute,longitute,country
323403,1514030602625990657,734192770664009728,@Breaking911 Money for Ukraine. No protection ...,en,2022-04-12,"Apollo Beach, FL",money for ukraine no protection for our bord...,Apollo Beach,money for ukraine no protection for our bord...,27.772945,-82.398138,United States
323404,1514030602210664454,791752212267532288,Here’s why Boeing just scrapped dozens of jet ...,en,2022-04-12,USA,here s why boeing just scrapped dozens of jet ...,USA,here s why boeing just scrapped dozens of jet ...,39.783730,-100.445882,United States
323405,1514030601510268928,791752212267532288,Djokovic collapses in shocking upset https://t...,en,2022-04-12,USA,djokovic collapses in shocking upset,USA,djokovic collapses in shocking upset,39.783730,-100.445882,United States
323406,1514030600642265089,773169220855136256,A German State Is Last in Almost Everything. B...,en,2022-04-12,"Cambridge, MA, USA",a german state is last in almost everything b...,USA,a german state is last in almost everything b...,39.783730,-100.445882,United States
323407,1514030600218419204,889963693,@CelticWitch7 @GlobalTotty @Femi_Sorry @Jacob_...,en,2022-04-12,"East Midlands, England",if you read up on the underlying issues as...,England,if you read up on the underlying issues as...,52.531021,-1.264906,United Kingdom


In [40]:
df_copy = df.copy()

In [41]:
df = df.dropna()
df['country']= df['country'].apply(lambda x : x.strip() )  
result = df.groupby(['country','lang']).size().reset_index(name='count') 
result

/Users/kaixuanchin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,country,lang,count
0,Afghanistan,de,1
1,Afghanistan,en,49
2,Afghanistan,es,1
3,Afghanistan,fa,1
4,Afghanistan,fr,1
...,...,...,...
1350,Zimbabwe,in,5
1351,Zimbabwe,pt,1
1352,Zimbabwe,ro,1
1353,Zimbabwe,tl,3


In [44]:
only_english = result.loc[result['lang'] == 'de']
only_english.sort_values(by=['count'], ascending=False)

,country,lang,count
381,Germany,de,6981
1271,United States,de,561
56,Austria,de,466
1234,United Kingdom,de,236
1092,Switzerland,de,219
...,...,...,...
581,Jamaica,de,1
604,Jersey,de,1
610,Kazakhstan,de,1
695,Mali,de,1


In [48]:
import ast 

cardiff_label = []
cardiff_score = []
with open('../app/static/files/output/cardiff_result.txt', 'r') as f:
    for line in f:
        line = line.strip()
        data = ast.literal_eval(line)
        if data.get('label') == 'LABEL_0': 
            label = 0
        elif data.get('label') == 'LABEL_1':
            label = 1 
        else:
            label = 2 
        cardiff_label.append(label)
        cardiff_score.append(data.get('score'))
df_copy['cardiff_label'] = cardiff_label
df_copy['cardiff_score'] = cardiff_score

In [49]:
df_copy.head()

,tweetId,author_id,tweet,lang,created_at,user_loc,clean_tweet,extracted_user_loc,translated_text,latitute,longitute,country,cardiff_label,cardiff_score
323403,1514030602625990657,734192770664009728,@Breaking911 Money for Ukraine. No protection ...,en,2022-04-12,"Apollo Beach, FL",money for ukraine no protection for our bord...,Apollo Beach,money for ukraine no protection for our bord...,27.772945,-82.398138,United States,0,0.940989
323404,1514030602210664454,791752212267532288,Here’s why Boeing just scrapped dozens of jet ...,en,2022-04-12,USA,here s why boeing just scrapped dozens of jet ...,USA,here s why boeing just scrapped dozens of jet ...,39.783730,-100.445882,United States,1,0.564979
323405,1514030601510268928,791752212267532288,Djokovic collapses in shocking upset https://t...,en,2022-04-12,USA,djokovic collapses in shocking upset,USA,djokovic collapses in shocking upset,39.783730,-100.445882,United States,0,0.535684
323406,1514030600642265089,773169220855136256,A German State Is Last in Almost Everything. B...,en,2022-04-12,"Cambridge, MA, USA",a german state is last in almost everything b...,USA,a german state is last in almost everything b...,39.783730,-100.445882,United States,1,0.498729
323407,1514030600218419204,889963693,@CelticWitch7 @GlobalTotty @Femi_Sorry @Jacob_...,en,2022-04-12,"East Midlands, England",if you read up on the underlying issues as...,England,if you read up on the underlying issues as...,52.531021,-1.264906,United Kingdom,0,0.588066


In [50]:
df = df_copy.copy()

In [51]:
df = df.dropna()
df['country']= df['country'].apply(lambda x : x.strip() )  
result = df.groupby(['country','lang']).size().reset_index(name='count') 
result

,country,lang,count
0,Afghanistan,de,1
1,Afghanistan,en,49
2,Afghanistan,es,1
3,Afghanistan,fa,1
4,Afghanistan,fr,1
...,...,...,...
1350,Zimbabwe,in,5
1351,Zimbabwe,pt,1
1352,Zimbabwe,ro,1
1353,Zimbabwe,tl,3


In [52]:
only_english = result.loc[result['lang'] == 'de']
only_english.sort_values(by=['count'], ascending=False)

,country,lang,count
381,Germany,de,6981
1271,United States,de,561
56,Austria,de,466
1234,United Kingdom,de,236
1092,Switzerland,de,219
...,...,...,...
581,Jamaica,de,1
604,Jersey,de,1
610,Kazakhstan,de,1
695,Mali,de,1
